In [20]:
from steam_profile_games import obtener_juegos_steam
import pandas as pd

In [ ]:
API_KEY = '5EF8885FBA34D73C53DD4AF7564C44C7'

# URL del perfil de Steam
perfil_url = 'https://steamcommunity.com/id/MetalMaxALIZ/'

# Obtener el DataFrame con todos los juegos
df_juegos = obtener_juegos_steam(perfil_url, API_KEY)

# Mostrar información
print(f"Total de juegos: {len(df_juegos)}")
df_juegos.head(10)

Total de juegos: 473


,app_id,nombre,tiempo_juego_minutos,img_icon_url,tiempo_windows_minutos,tiempo_mac_minutos,tiempo_linux_minutos,playtime_deck_forever,rtime_last_played,content_descriptorids,playtime_disconnected,has_community_visible_stats,has_leaderboards,tiempo_juego_horas,imagen_url,store_url
0,10,Counter-Strike,7707,6b0312cda02f5f777efa2f3318c307ff9acafbb5,0,0,0,0,1475882850,"[2, 5]",0,NaN,NaN,128.450000,https://cdn.cloudflare.steamstatic.com/steam/a...,https://store.steampowered.com/app/10
1,30,Day of Defeat,5,aadc0ce51ff6ba2042d633f8ec033b0de62091d0,0,0,0,0,86400,"[2, 5]",0,NaN,NaN,0.083333,https://cdn.cloudflare.steamstatic.com/steam/a...,https://store.steampowered.com/app/30
2,40,Deathmatch Classic,20,c525f76c8bc7353db4fd74b128c4ae2028426c2a,0,0,0,0,86400,NaN,0,NaN,NaN,0.333333,https://cdn.cloudflare.steamstatic.com/steam/a...,https://store.steampowered.com/app/40
3,60,Ricochet,6,98c69e04cd59b838e05cb6980c12c05874c6419e,0,0,0,0,86400,NaN,0,NaN,NaN,0.100000,https://cdn.cloudflare.steamstatic.com/steam/a...,https://store.steampowered.com/app/60
4,80,Counter-Strike: Condition Zero,102,077b050ef3e89cd84e2c5a575d78d53b54058236,0,0,0,0,86400,"[2, 5]",0,NaN,NaN,1.700000,https://cdn.cloudflare.steamstatic.com/steam/a...,https://store.steampowered.com/app/80
5,100,Counter-Strike: Condition Zero Deleted Scenes,0,077b050ef3e89cd84e2c5a575d78d53b54058236,0,0,0,0,0,NaN,0,NaN,NaN,0.000000,https://cdn.cloudflare.steamstatic.com/steam/a...,https://store.steampowered.com/app/100
6,550,Left 4 Dead 2,507,7d5a243f9500d2f8467312822f8af2a2928777ed,0,0,0,0,1422303949,"[2, 5]",0,True,NaN,8.450000,https://cdn.cloudflare.steamstatic.com/steam/a...,https://store.steampowered.com/app/550
7,570,Dota 2,2553,0bbb630d63262dd66d2fdd0f7d37e8661a410075,37,0,0,0,1664845515,[5],0,NaN,NaN,42.550000,https://cdn.cloudflare.steamstatic.com/steam/a...,https://store.steampowered.com/app/570
8,620,Portal 2,376,25a5a16b2423bf7487ac5340b5b0948cef48c5f8,0,0,0,0,1498957512,NaN,0,True,True,6.266667,https://cdn.cloudflare.steamstatic.com/steam/a...,https://store.steampowered.com/app/620
9,730,Counter-Strike 2,37285,8dbc71957312bbd3baea65848b545be9eae2a355,9663,0,0,0,1704987692,"[2, 5]",0,True,NaN,621.416667,https://cdn.cloudflare.steamstatic.com/steam/a...,https://store.steampowered.com/app/730


In [11]:
# Nos quedamos solo con las columnas de interés 
columnas_interes = ['app_id', 'nombre', 'tiempo_juego_horas', 'imagen_url', 'store_url']
df_juegos_interes = df_juegos[columnas_interes]
df_juegos_interes.shape


(473, 5)

In [ ]:
# Eliminamos los que tengan el mismo nombre
df_juegos_interes = df_juegos_interes.drop_duplicates(subset=['nombre'])

In [4]:
# Crear un dataframe de juegos_rawg.db
from rawg_api_scraper import obtener_dataframe_juegos
df_rawg = obtener_dataframe_juegos()
df_rawg.shape

(426474, 7)

In [5]:
# Eliminar todas las filas en las que tags no contenga Steam Achievements o Steam Cloud
df_rawg_filtrado = df_rawg[df_rawg['tags'].str.contains('Steam Achievements|Steam Cloud', na=False)]
df_rawg_filtrado

,id,name,background_image,metacritic,genres,tags,fecha_extraccion
0,1,D/Generation HD,https://media.rawg.io/media/screenshots/5c4/5c...,NaN,"Adventure, Puzzle","Singleplayer, Steam Achievements, Full control...",2025-11-22 13:21:39
1,2,Extreme Exorcism,https://media.rawg.io/media/screenshots/67d/67...,75.0,"Action, Adventure, Casual, Indie","Singleplayer, Steam Achievements, Multiplayer,...",2025-11-22 13:14:52
2,3,Toto Temple Deluxe,https://media.rawg.io/media/screenshots/0af/0a...,NaN,"Action, Casual, Indie","Singleplayer, Steam Achievements, Multiplayer,...",2025-11-22 13:22:41
3,4,Penarium,https://media.rawg.io/media/screenshots/68b/68...,80.0,"Action, Arcade, Casual, Indie","Singleplayer, Steam Achievements, Full control...",2025-11-22 13:13:46
4,5,Standpoint,https://media.rawg.io/media/screenshots/238/23...,NaN,"Action, Indie","Singleplayer, Steam Achievements, Full control...",2025-11-22 13:41:02
...,...,...,...,...,...,...,...
426463,1015065,Clueless: An Item Finder,https://media.rawg.io/media/screenshots/fea/fe...,NaN,Casual,"Singleplayer, Steam Achievements, First-Person...",2025-11-22 15:04:01
426464,1015066,Total Chaos (itch.io),https://media.rawg.io/media/screenshots/963/96...,NaN,"Action, Adventure, Indie","Singleplayer, Steam Achievements, Full control...",2025-11-22 14:35:04
426465,1015067,Tales In Text: Multiplayer Immersive Adventures,https://media.rawg.io/media/screenshots/566/56...,NaN,"Adventure, Casual, Indie","Singleplayer, Steam Achievements, Multiplayer,...",2025-11-22 15:04:01
426466,1015068,Cult of the Child Eater,https://media.rawg.io/media/screenshots/961/96...,NaN,"Action, Adventure, Indie","Singleplayer, Steam Achievements, Multiplayer,...",2025-11-22 15:04:01


In [28]:
# Eliminamos la columna fecha_extraccion
df_rawg_filtrado = df_rawg_filtrado.drop(columns=['fecha_extraccion'])

In [17]:
# Creamos un dataframe de steam_data.db
from SteamSpy import obtener_dataframe_steam_data
df_steam_spy = obtener_dataframe_steam_data()
df_steam_spy

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,730,Counter-Strike: Global Offensive,Valve,Valve,,7642084,1173003,0,"100,000,000 .. 200,000,000",33076,731,6137,295,0,0,0,1013936
1,1172470,Apex Legends,Respawn,Electronic Arts,,668053,326926,0,"100,000,000 .. 200,000,000",10632,622,962,250,0,0,0,124262
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",,1520457,1037487,0,"100,000,000 .. 200,000,000",23472,855,6057,316,0,0,0,314682
3,1623730,Palworld,Pocketpair,Pocketpair,,358266,22443,0,"50,000,000 .. 100,000,000",3799,689,2202,219,2999,2999,0,18028
4,440,Team Fortress 2,Valve,Valve,,1044264,117208,0,"50,000,000 .. 100,000,000",23714,330,4326,89,0,0,0,43819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86536,2571600,Meurtre Au Florian,"Nikoballz, PnDAa",Nikoballz,,7,2,0,"0 .. 20,000",0,0,0,0,499,499,0,0
86537,3223680,Rescue Girl Operation: Code X,2024-NGame,2024-NGame,,1,1,0,"0 .. 20,000",0,0,0,0,399,399,0,0
86538,3310110,Big Burger Tower,Frozen Logic Studios,Frozen Logic Studios,,2,1,0,"0 .. 20,000",0,0,0,0,399,399,0,0
86539,1589920,Long Ago: A Puzzle Tale,GrimTalin,GrimTalin,,15,2,0,"0 .. 20,000",0,0,0,0,499,499,0,0


In [21]:
# Nos quedamos solo con las columnas de interés 
columnas_interes_steam_spy = ['name', 'developer','positive','negative', 'owners', 'price', 'initialprice', 'discount']
df_steam_spy_interes = df_steam_spy[columnas_interes_steam_spy]
df_steam_spy_interes.shape

(86541, 8)

In [ ]:
# Borramos los datos incoherentes como los precios = None




In [29]:
# combinar los dataset df_steam_spy y df_rawg_filtrado usando la columna name
df_combinado = pd.merge(df_steam_spy_interes, df_rawg_filtrado, left_on='name', right_on='name', how='inner')
df_combinado.shape

(33264, 13)

In [ ]:
df_combinado

,name,developer,positive,negative,owners,price,initialprice,discount,id,background_image,metacritic,genres,tags
0,Counter-Strike: Global Offensive,Valve,7642084,1173003,"100,000,000 .. 200,000,000",0,0,0,4291,https://media.rawg.io/media/games/736/73619bd3...,81.0,Shooter,"Steam Achievements, Multiplayer, Full controll..."
1,Team Fortress 2,Valve,1044264,117208,"50,000,000 .. 100,000,000",0,0,0,11859,https://media.rawg.io/media/games/46d/46d98e69...,92.0,"Action, Shooter","Steam Achievements, Multiplayer, steam-trading..."
2,Call of Duty: Modern Warfare II,"Treyarch, Raven Software, Beenox, High Moon St...",419594,294520,"50,000,000 .. 100,000,000",3849,6999,45,791636,https://media.rawg.io/media/games/1af/1af8edc7...,NaN,"Action, Shooter","Singleplayer, Steam Achievements, Multiplayer,..."
3,Left 4 Dead 2,Valve,940221,23762,"50,000,000 .. 100,000,000",999,999,0,12020,https://media.rawg.io/media/games/d58/d588947d...,89.0,"Action, Shooter","Singleplayer, Steam Achievements, Multiplayer,..."
4,Unturned,Smartly Dressed Games,506516,48852,"50,000,000 .. 100,000,000",0,0,0,9743,https://media.rawg.io/media/games/18c/18ca24ed...,NaN,"Action, Shooter","Singleplayer, Steam Achievements, Multiplayer,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33259,Call of Duty: Black Ops III,"Treyarch, Aspyr (Mac)",38969,23378,"0 .. 20,000",1979,5999,67,906,https://media.rawg.io/media/games/fd6/fd6a1eec...,73.0,"Action, Shooter","Singleplayer, Steam Achievements, Multiplayer,..."
33260,101 Cats Hidden in New Delhi,NaipSoft,0,0,"0 .. 20,000",99,99,0,996968,https://media.rawg.io/media/screenshots/315/31...,NaN,"Casual, Indie","Singleplayer, Steam Achievements, Steam Cloud,..."
33261,Mycelium Heaven,3DGoblinDev,33,3,"0 .. 20,000",999,999,0,974466,https://media.rawg.io/media/screenshots/cc3/cc...,NaN,"Casual, Indie, Puzzle","Singleplayer, Steam Achievements, Full control..."
33262,The Game Store,echomune,6,4,"0 .. 20,000",799,799,0,861372,https://media.rawg.io/media/screenshots/c25/c2...,NaN,"Simulation, Indie","Singleplayer, Steam Achievements, Full control..."


In [37]:
# Cambiamos la columna 'price' y 'initialprice' a tipo int

#df_combinado['price'] = df_combinado['price'].astype(int) / 100
#df_combinado['initialprice'] = df_combinado['initialprice'].astype(int) / 100